# Parser Prototype

In [1]:
from lark import Lark

with open("grammar.lark") as grammar_file:
    grammar = grammar_file.read()

In [2]:
# vm, iface = get_consts(smtsorts, ["vm", "iface"])
# return And(
#     smtenc.element_class_fun(vm) == smtenc.classes["infrastructure_VirtualMachine"],
#     Not(
#         Exists(
#             [iface],
#             ENCODINGS.association_rel(vm, smtenc.associations["infrastructure_ComputingNode::ifaces"], iface)
#         )
#     )
# )

expr_to_parse = r"""
-   "example requirement to test"
    # Expr to parse
    not vm is class infrastructure.VirtualMachine
    and
    vm is not class infrastructure.Storage
    iff
    not exists iface, apple (
        (vm has association infrastructure.ComputingNode->ifaces iface)
        or
        (vm has attribute infrastructure.ComputingNode->os Os1)
        and
        (vm has attribute infrastructure.ComputingNode->memory_mb 1024)
        and
        (vm has attribute infrastructure.ComputingNode->architecture "linux")
        and
        (vm has attribute application.SoftwareComponent->isPersistent true)
    )
    ---
    "Virtual Machine {vm} has no iface"

+   "example requirement to test"
    # Expr to parse
    not vm is class infrastructure.VirtualMachine
    and
    vm is not class infrastructure.Storage
    iff
    not exists iface, apple (
        (vm has association infrastructure.ComputingNode->ifaces iface)
        or
        (vm has attribute infrastructure.ComputingNode->os Os1)
    )
    ---
    "Virtual Machine {vm} has no iface"
"""

In [3]:
parser = Lark(grammar, start="requirements")
tree = parser.parse(expr_to_parse)

print(tree.pretty())

requirements
  requirement
    -
    req_name	"example requirement to test"
    iff_expr
      and_expr
        negation
          equality
            const_or_class	vm
            const_or_class	infrastructure.VirtualMachine
        inequality
          const_or_class	vm
          const_or_class	infrastructure.Storage
      negation
        exists
          bound_consts
            iface
            apple
          and_expr
            or_expr
              association_expr
                vm
                infrastructure.ComputingNode->ifaces
                iface
              attribute_expr
                vm
                infrastructure.ComputingNode->os
                value	Os1
            attribute_expr
              vm
              infrastructure.ComputingNode->memory_mb
              value	1024
            attribute_expr
              vm
              infrastructure.ComputingNode->architecture
              value	"linux"
            attribute_expr
              vm
      

We need the `ModelChecker` to import `SMTEncodings` and `SMTSorts` in order to create our Z3 constants programmatically.

Now the model checker should expose the *intermediate model checker* which should provide us with those two collections.

In [4]:
from mc_openapi.doml_mc import ModelChecker, DOMLVersion
from mc_openapi.doml_mc.imc import IntermediateModelChecker

# Import DOMLX as bytes
doml_document_path = "../../../tests/doml/faas.domlx"
with open(doml_document_path, "rb") as xmif:
    doml_xmi = xmif.read()

model_checker = ModelChecker(doml_xmi, DOMLVersion.V2_0)

intermediate_model_checker = IntermediateModelChecker(
    model_checker.metamodel,
    model_checker.inv_assoc,
    model_checker.intermediate_model
)

The parser will now produce a Z3 expression to evaluate.

In [5]:
from pprint import pprint
from z3 import Not, And, Or, Xor, Implies, Exists, ForAll

from mc_openapi.doml_mc.dsl_parser.parser import Parser

parser = Parser(grammar)

reqs_store, user_value_strings = parser.parse(expr_to_parse)

pprint(reqs_store.get_all_requirements())

[Requirement(assert_callable=<function DSLTransformer.iff_expr.<locals>.<lambda> at 0x7f5060012e60>,
             assert_name='example_requirement_to_test',
             description='example requirement to test',
             error_description=<function DSLTransformer.requirement.<locals>.<lambda> at 0x7f50600128c0>,
             flipped=True),
 Requirement(assert_callable=<function DSLTransformer.iff_expr.<locals>.<lambda> at 0x7f5060011fc0>,
             assert_name='example_requirement_to_test',
             description='example requirement to test',
             error_description=<function DSLTransformer.requirement.<locals>.<lambda> at 0x7f5060012320>,
             flipped=True)]


In [6]:
intermediate_model_checker.instantiate_solver(user_value_strings)
ENCODINGS =  intermediate_model_checker.smt_encoding
SORTS = intermediate_model_checker.smt_sorts

assert ENCODINGS and SORTS

In [7]:
for req in reqs_store.get_all_requirements():
    print(req.assert_callable(ENCODINGS, SORTS))

And(Not(elem_class(vm) == infrastructure_VirtualMachine),
    elem_class(vm) != infrastructure_Storage) ==
Not(Exists([iface, apple],
           And(Or(association(vm,
                              infrastructure_ComputingNode::ifaces,
                              iface),
                  attribute(vm,
                            infrastructure_ComputingNode::os,
                            Os1)),
               attribute(vm,
                         infrastructure_ComputingNode::memory_mb,
                         int(1024)),
               attribute(vm,
                         infrastructure_ComputingNode::architecture,
                         ss(ss_5__linux_)),
               attribute(vm,
                         application_SoftwareComponent::isPersistent,
                         bool(True)))))
And(Not(elem_class(vm) == infrastructure_VirtualMachine),
    elem_class(vm) != infrastructure_Storage) ==
Not(Exists([iface, apple],
           Or(association(vm,
                    